In [1]:
import os
import glob
import tqdm
import torch
import random
import librosa
import argparse
import numpy as np
from multiprocessing import Pool, cpu_count

from utils.audio import Audio
from utils.hparams import HParam
import utils.mixer as mx


def formatter(dir_, form, num):
    return os.path.join(dir_, form.replace('*', '%06d' % num))

def vad_merge(w):
    intervals = librosa.effects.split(w, top_db=20)
    temp = list()
    for s, e in intervals:
        temp.append(w[s:e])
    return np.concatenate(temp, axis=None)

In [2]:
from generator import mix

if __name__ == '__main__':
    #parser = argparse.ArgumentParser()
    #parser.add_argument('-c', '--config', type=str, required=True,
    #                    help="yaml file for configuration")
    #args = parser.parse_args()

    #hp = HParam(args.config)
    hp = HParam('/home/kiosk/git/VFWS/config/config.yaml')
    
    vfws_dir = hp.data.vfws_dir
    
    clean_dir = os.path.join(vfws_dir,'clean')
    mixed_dir = os.path.join(vfws_dir,'mixed')
    
    os.makedirs(clean_dir, exist_ok=True)
    os.makedirs(mixed_dir, exist_ok=True)
    
    os.makedirs(os.path.join(clean_dir, 'train'), exist_ok=True)
    os.makedirs(os.path.join(clean_dir, 'test'), exist_ok=True)
    os.makedirs(os.path.join(mixed_dir, 'train'), exist_ok=True)
    os.makedirs(os.path.join(mixed_dir, 'test'), exist_ok=True)

    cpu_num = cpu_count()
    
    norm_dir = hp.data.norm_dir
    
    #train_spk = [x for x in glob.glob(os.path.join(norm_dir, 'train','**'), recursive=True) if not os.path.isdir(x)]
    #train_spk = [x for x in train_spk if len(x) >= 2]
    
    test_spk = [x for x in glob.glob(os.path.join(norm_dir, 'dev','**'), recursive=True) if not os.path.isdir(x)]
    test_spk = [x for x in test_spk if len(x) >= 2]

    audio = Audio(hp)

    noise_files = [x for x in glob.glob(os.path.join(hp.data.noise_dir,'**'), recursive=True) if not os.path.isdir(x)]
    def train_wrapper(num):
        target = train_spk[num]
        noise_idx = np.random.randint(len(noise_files))
        noise = noise_files[noise_idx]
        mix(hp, audio, target, noise, num, train=True)

    def test_wrapper(num):
        target = test_spk[num]
        noise_idx = np.random.randint(len(noise_files))
        noise = noise_files[noise_idx]
        mix(hp, audio, target, noise, num, train=False)

    #arr = list(range(len(train_spk)))
    #with Pool(cpu_num) as p:
    #    r = list(tqdm.tqdm(p.imap(train_wrapper, arr), total=len(arr)))

    arr = list(range(len(test_spk)))
    with Pool(cpu_num) as p:
        r = list(tqdm.tqdm(p.imap(test_wrapper, arr), total=len(arr)))


/home/nas/user/kbh/VFWS/pre_data_librispeech/utils/hparams.py:18: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  for doc in docs:
  1%|          | 16/2703 [00:09<27:37,  1.62it/s] 


KeyboardInterrupt: 